'''
https://www.youtube.com/watch?v=wiGkV37Kbxk

math.factorial(), math.comb(), math.perm()  
np.sum(), math.fsum()  
mean(), stdev(), pstdev(). variance(). pvariance, in statistics  
math.hypot()  
'''

In [2]:
!python -V

Python 3.10.11


In [1]:
import math 
import numpy as np

#### Factorial goes faster

In [39]:
# basic factorial 
%timeit math.prod(range(1, 150))

10.6 µs ± 32.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [33]:
math.prod(range(1,6))

120

In [34]:
def f(x):
    return x * f(x-1) if x > 1 else 1

In [28]:
f(5)

120

In [40]:
%timeit math.factorial(150)

1.8 µs ± 3.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [44]:
(1.8/10.6)

0.169811320754717

#### compute factorials like a boss  

Winning ideas:  
Left shift is cheaper than multiplying by two   
Pulling out events leaves recurring odd factories  
Dynamic programming reuses previously computed odd factorials  

In [75]:
s = ""
so = ""
se = ""
for i in range(1,20):
    s += str(i) + '*'
    if i%2 == 0:
        se += str(i) + '*'
    else: 
        so += str(i) + '*'
print(s, so, se)

1*2*3*4*5*6*7*8*9*10*11*12*13*14*15*16*17*18*19* 1*3*5*7*9*11*13*15*17*19* 2*4*6*8*10*12*14*16*18*


In [71]:
1*2*3*4*5*6*7*8*9*10*11*12*13*14*15*16*17*18*19

121645100408832000

In [79]:
# collect even terms, next divide each even by 2
1*3*5*7*9*11*13*15*17*19 *2*4*6*8*10*12*14*16*18

121645100408832000

In [81]:
# divide terms by two and replace with left shift
1*3*5*7*9*11*13*15*17*19 *1*2*3*4*5*6*7*8*9 << 9

121645100408832000

In [84]:
1*3*5*7*9*11*13*15*17*19 *1*3*5*7*9 * 1*2*3*4 <<13

121645100408832000

In [86]:
1*3*5*7*9*11*13*15*17*19 *1*3*5*7*9 * 1*3 *1*2 <<15

121645100408832000

In [88]:
# replace even term with left shift
1*3*5*7*9*11*13*15*17*19 *1*3*5*7*9 * 1*3 <<16

121645100408832000

In [93]:
# factor-out common subsequences and replace with powers 
(1*3)**3 *(5*7*9)**2 *(11*13*15*17*19)**1 <<16

121645100408832000

#### combinations got faster

In [52]:
# combinations of 52 items, taking out 31 at a time 
%timeit  math.factorial(52) / (math.factorial(31) * math.factorial(21))

1.69 µs ± 6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [53]:
%timeit math.comb(52, 31)

2.26 µs ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### summation became accurate

In [54]:
data  = [0.1] * 10

In [56]:
# result is wrong
sum(data)

0.9999999999999999

In [59]:
# result is correct
np.sum(data)

1.0

In [60]:
# result is correct, accurate floating point sum 
math.fsum(data)

1.0

####  Hypot became multi-dimensional and  accurate

In [61]:
math.hypot(3.64142839394477e+179,
          -6.10824106882006e+181,
          1.050035987292735e+181)

6.197944056661425e+181

In [96]:
from math import prod

def py_fact(n):
    m = 1 
    odd_fact = 1
    result = 1 
    for s in reversed(range(n.bit_length() - 1)):
        new_m = (ec := n >> s) - (~ec & 1)
        odd_fact *= prod(range(m + 2, new_m + 2, 2))
        result *= odd_fact
        m = new_m
    shift = sum(n >> i for i in range(1, n.bit_length()))
    return result << shift 

In [97]:
py_fact(19)

121645100408832000

In [98]:
from math import factorial
factorial(19)

121645100408832000

#### Compute combinations like a champ

In [99]:
from math import comb, factorial, log, exp, lgamma

In [100]:
n = 52
k = 31
comb(n, k)

191991813933920

#### The traditional approach  
This works just fine. However the intermediate values are huge.

In [103]:
def baseline(n, k):
    return factorial(n) // (factorial(k) * factorial(n-k))

In [104]:
baseline(n, k)

191991813933920

In [105]:
# Intermediate values are large numbers 

In [108]:
# the implementation hides intermedite values, very huge numbers
def baseline(n, k):
    return factorial(n) // (factorial(k) * factorial(n-k))

In [109]:
baseline(n, k)

191991813933920

In [113]:
# Intermediate values are large numbers printed below
# big numbers mean slow

In [114]:
print(f'{factorial(n) =:,}')
print(f'{factorial(k) =:,}')
print(f'{factorial(n-k) =:,}')
print(f'{factorial(n) =:,}')
print(f'{factorial(k) * factorial(n-k) =: }')

factorial(n) =80,658,175,170,943,878,571,660,636,856,403,766,975,289,505,440,883,277,824,000,000,000,000
factorial(k) =8,222,838,654,177,922,817,725,562,880,000,000
factorial(n-k) =51,090,942,171,709,440,000
factorial(n) =80,658,175,170,943,878,571,660,636,856,403,766,975,289,505,440,883,277,824,000,000,000,000
factorial(k) * factorial(n-k) = 420112574167901332880138092414311435809587200000000000


#### Log gamma approach  
Logarithms tame the large values, but the Lanczos formula still takes a lot of steps

In [117]:
def lg_comb(n, k):
    return round(exp(lgamma(n+1) - (lgamma(k+1) + lgamma(n-k+1))))

In [118]:
lg_comb(n, k)

191991813933925

In [119]:
print(lgamma(n+1))
print(lgamma(k+1))
print(lgamma(n-k+1))

156.3608363030788
78.0922235533153
45.38013889847691


#### Log factorial table  
This is fast and compact, but the accuracy falters at around n = 53  

In [140]:
# Precomputation  , lookup can be several times faster
log_fact = {n : log(factorial(n)) for n in range(54)}

In [141]:
def lf_comb(n, k):
    return round(exp(log_fact[n] - (log_fact[k] + log_fact[n-k])))

In [142]:
lf_comb(n, k)

191991813933925

In [143]:
print(log_fact[n])
print(log_fact[k])
print(log_fact[n-k])



156.3608363030788
78.0922235533153
45.38013889847691


In [144]:
# group theory

In [145]:
# precomputation of the lookup table
odd_fact = []
shift_fact = []
inv_odd_fact = []

for nn in range(128):
    fact = factorial(nn)
    
    shift  = (fact & -fact).bit_length() - 1
    odd = fact >> shift
    inv = pow(odd, -1, 2**64)
    
    odd_fact.append(odd % 2**64)
    shift_fact.append(shift)
    inv_odd_fact.append(inv)

In [153]:
# one of the fastest thing in python 
# total 6 operations, nano seconds , factor of 15 improvement 
def fast_comb(n, k):
    odd = odd_fact[n] * inv_odd_fact[k] * inv_odd_fact[n-k] % 2**64
    shift = shift_fact[n] - shift_fact[k] - shift_fact[n-k]
    return odd << shift

In [154]:
%timeit fast_comb(n, k)

809 ns ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [155]:
print(factorial(n))
print(factorial(k))
print(factorial(n-k))

80658175170943878571660636856403766975289505440883277824000000000000
8222838654177922817725562880000000
51090942171709440000


In [156]:
print(odd_fact[n])
print(inv_odd_fact[k])
print(inv_odd_fact[n-k])

16558329919276434777
1594080273878023779
4287997350378269393


In [157]:
def big_comb(n, k):
    if k == 0:
        return 1
    if k == 1:
        return n
    if n <=67:
        return fast_comb(n, k)
    j = k // 2
    return big_comb(n, j) * big_comb(n-j,k-j) // big_comb(k, j)

In [158]:
# easy to compute large number quickly
big_comb(1001, 317)

713329781083668697436467130320371383828565538699572818948873472006630273863867326544975139763804851249232258863022323651287391333212191519801473487538142922543743781969447992148066066292868270307638459872816543654297060122680568457501326587479365018024349380635646362000

In [159]:
# sum, round accuracy
# 

In [161]:
# add consecutive adds, rms error adds up 
def regular_sum(data):
    total = 0.0
    for x in data:
        total += x
    return total

In [162]:
# sorted addition
def sorted_sum(data):
    return regular_sum(sorted(data, key=abs))

In [183]:
# pairwise addition, get rid of most of errors
def pairwise(data):
    n = len(data)
    if n <= 2:
        return regular_sum(data)
    m = n // 2 
    return pairwise(data[:m]) + pairwise(data[m:])

In [178]:
# kahan compensated addition
def kahan(data):
    total = 0.0
    c = 0.0
    for x in data:
        y = x - c 
        t = total + y
        c = (t - total) - y
        total = t
    return total

In [192]:
# neumaier variant
def neumaier(data):
    total = 0.0
    c = 0.0
    for x in data:
        t = total + x
        if abs(total) >= abs(x):
            c += (total - t) + x
        else:
            c += (x - t) + total
        total = t
    return total + c

In [190]:
# multiple paritals
# this algorithm appears in the standard library as math.fsum()
def fsum(iterable):
    partials = []
    for x in iterable:
        i = 0
        for y in partials:
            if abs(x) < abs(y):
                x, y = y, x
            hi = x + y
            lo = y - (hi - x)
            if lo:
                partials[i] = lo
                i += 1 
            x = hi
        partials[i:] = [x]
    return regular_sum(partials)

In [194]:
from fractions import Fraction

def fr_sum(data):
    return sum(map(Fraction, data))

In [197]:
from random import expovariate, random

data = [expovariate(1.0) - random() / 2.0 for i in range(8)]
for method in regular_sum, sorted_sum, pairwise, kahan, neumaier, fsum, type:
    print(method(data), method.__name__)

8.109049114001557 regular_sum
8.109049114001557 sorted_sum
8.109049114001555 pairwise
8.109049114001557 kahan
8.109049114001557 neumaier
8.109049114001557 fsum
<class 'list'> type
